<a href="https://colab.research.google.com/github/VlasovArs/Recommendation_systems/blob/main/1_The_simplest_methods_of_building_recommendation_systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Рекомендательная система на библиотеке surprise

Сайт библиотеки: http://surpriselib.com/

Установки библиотеки:
```
$ pip install scikit-surprise```
или
```
$ conda install -c conda-forge scikit-surprise```

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("recdemo.csv", sep=";") #небольшой dataset

In [ ]:
df

,id,A,B,C,D,E,F
0,1,3.0,4.0,2.0,4.0,1.0,NaN
1,2,3.0,4.0,2.0,4.0,NaN,2.0
2,3,NaN,2.0,5.0,5.0,NaN,5.0
3,4,NaN,NaN,NaN,NaN,4.0,NaN
4,5,3.0,NaN,2.0,4.0,4.0,NaN
5,6,NaN,5.0,5.0,5.0,NaN,5.0
6,7,1.0,NaN,NaN,2.0,NaN,3.0
7,8,NaN,NaN,NaN,NaN,4.0,4.0


In [ ]:
df_unpivot=pd.melt(df, id_vars=['id'])
df_unpivot.head()

,id,variable,value
0,1,A,3.0
1,2,A,3.0
2,3,A,NaN
3,4,A,NaN
4,5,A,3.0


In [ ]:
df_unpivot.dropna(inplace=True)
df_unpivot.columns=['userID', 'itemID', 'rating']

In [ ]:
from surprise import Dataset
from surprise import Reader

reader = Reader(rating_scale=(1, 5)) # Зададим разброс оценок
data = Dataset.load_from_df(df_unpivot[['userID', 'itemID', 'rating']], reader) #создадим объект, с которым умеет работать библиотека

In [ ]:
trainset = data.build_full_trainset()
testset = trainset.build_anti_testset()

In [ ]:
testset[0:10]

[(1, 'F', 3.4642857142857144),
 (2, 'E', 3.4642857142857144),
 (5, 'B', 3.4642857142857144),
 (5, 'F', 3.4642857142857144),
 (7, 'B', 3.4642857142857144),
 (7, 'C', 3.4642857142857144),
 (7, 'E', 3.4642857142857144),
 (3, 'A', 3.4642857142857144),
 (3, 'E', 3.4642857142857144),
 (6, 'A', 3.4642857142857144)]

In [ ]:
samples = [[0., 0., 0.], [0., .5, 0.], [1., 1., .5], [0.3, .5, 0.2], [.2, 1., .5]]
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=2)
knn.fit(samples)
print(knn.kneighbors([[1., 1., 1.]]))

(array([[0.5       , 0.94339811]]), array([[2, 4]], dtype=int64))


In [ ]:
print(knn.kneighbors([[1., 0., 1.]]))

(array([[1.11803399, 1.17473401]]), array([[2, 3]], dtype=int64))


In [ ]:
from surprise import KNNBaseline
algo = KNNBaseline(k=1)
algo.fit(trainset)
predictions = algo.test(testset)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
predictions[0:5]

[Prediction(uid=1, iid='F', r_ui=3.4642857142857144, est=1.9577925900897966, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid=2, iid='E', r_ui=3.4642857142857144, est=1.0422074099102034, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid=5, iid='B', r_ui=3.4642857142857144, est=4.082661845573158, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid=5, iid='F', r_ui=3.4642857142857144, est=2.0826618455731576, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid=7, iid='B', r_ui=3.4642857142857144, est=3.8701641215302014, details={'actual_k': 1, 'was_impossible': False})]

In [ ]:
df_unpivot1 = df_unpivot.copy()
for i in predictions:
    df_unpivot1 = df_unpivot1.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est}, ignore_index=True)

In [ ]:
df_unpivot1.pivot(index='userID', columns='itemID', values='rating')

itemID,A,B,C,D,E,F
userID,,,,,,
1,3.000000,4.000000,2.000000,4.000000,1.000000,1.957793
2,3.000000,4.000000,2.000000,4.000000,1.042207,2.000000
3,3.311189,2.000000,5.000000,5.000000,4.090446,5.000000
4,3.072513,4.197382,2.072513,4.072513,4.000000,3.976639
5,3.000000,4.082662,2.000000,4.000000,4.000000,2.082662
6,3.469084,5.000000,5.000000,5.000000,4.248341,5.000000
7,1.000000,3.870164,1.870164,2.000000,3.691628,3.000000
8,3.095874,1.909554,2.095874,4.095874,4.000000,4.000000


In [ ]:
algo = KNNBaseline(k=3)
algo.fit(trainset)
# Than predict ratings for all pairs (u, i) that are NOT in the training set.
predictions = algo.test(testset)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
df_unpivot3 = df_unpivot.copy()
for i in predictions:
    df_unpivot3 = df_unpivot3.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est}, ignore_index=True)
df_unpivot3.pivot(index='userID', columns='itemID', values='rating')

itemID,A,B,C,D,E,F
userID,,,,,,
1,3.000000,4.000000,2.000000,4.000000,1.000000,2.507435
2,3.000000,4.000000,2.000000,4.000000,2.601745,2.000000
3,3.256100,2.000000,5.000000,5.000000,3.527693,5.000000
4,3.083864,4.197382,2.083864,4.083864,4.000000,3.976639
5,3.000000,3.834977,2.000000,4.000000,4.000000,3.013312
6,3.418235,5.000000,5.000000,5.000000,3.590444,5.000000
7,1.000000,3.366011,1.857717,2.000000,3.095321,3.000000
8,2.579863,3.471928,3.463240,4.463240,4.000000,4.000000


## Кросс-валидация

In [ ]:
from surprise.model_selection import cross_validate

In [ ]:
cross_validate(algo, data, cv=2, verbose=True)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 2 split(s).

                  Fold 1  Fold 2  Mean    Std     
RMSE (testset)    1.5068  1.4832  1.4950  0.0118  
MAE (testset)     1.2600  1.1369  1.1984  0.0615  
Fit time          0.00    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    


{'test_rmse': array([1.5067953 , 1.48324745]),
 'test_mae': array([1.25995309, 1.13694536]),
 'fit_time': (0.0, 0.000997304916381836),
 'test_time': (0.0, 0.0)}

In [ ]:
cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([1.52618574, 1.63112299, 1.42226037]),
 'fit_time': (0.0, 0.0, 0.0),
 'test_time': (0.000997304916381836, 0.0, 0.0)}

In [ ]:
for i in range(1,6):
    algo = KNNBaseline(k=i, verbose=False) #отключим вывод логирования
    cv=cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)
    print(str(i)+'NN:',np.mean(cv['test_rmse']))

1NN: 1.6649043410789377
2NN: 1.6636341764054048
3NN: 1.3534887218964426
4NN: 1.6999722513125624
5NN: 1.8018448597718197


## Item-based подход

In [ ]:
algo = KNNBaseline(k=5, verbose=False)
predictions = algo.fit(trainset).test(testset)
df_unpivot5 = df_unpivot.copy()
for i in predictions:
    df_unpivot5 = df_unpivot5.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est}, ignore_index=True)
df_unpivot5.pivot(index='userID', columns='itemID', values='rating')

itemID,A,B,C,D,E,F
userID,,,,,,
1,3.000000,4.000000,2.000000,4.000000,1.000000,2.810335
2,3.000000,4.000000,2.000000,4.000000,2.601745,2.000000
3,2.859638,2.000000,5.000000,5.000000,3.527693,5.000000
4,3.083864,4.197382,2.083864,4.083864,4.000000,3.976639
5,3.000000,3.918340,2.000000,4.000000,4.000000,3.239815
6,3.053741,5.000000,5.000000,5.000000,3.590444,5.000000
7,1.000000,3.566434,2.632849,2.000000,3.095321,3.000000
8,2.615467,3.529529,3.297505,4.043101,4.000000,4.000000


In [ ]:
sim_options = {'user_based': False } # Item-based подход

In [ ]:
algo = KNNBaseline(k=5,sim_options=sim_options, verbose=False)
predictions = algo.fit(trainset).test(testset)
df_unpivot5_items = df_unpivot.copy()
for i in predictions:
    df_unpivot5_items = df_unpivot5_items.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est}, ignore_index=True)
df_unpivot5_items.pivot(index='userID', columns='itemID', values='rating')

itemID,A,B,C,D,E,F
userID,,,,,,
1,3.000000,4.000000,2.000000,4.000000,1.000000,2.329176
2,3.000000,4.000000,2.000000,4.000000,2.376574,2.000000
3,3.858465,2.000000,5.000000,5.000000,4.678967,5.000000
4,3.818825,4.122894,3.956402,4.261502,4.000000,4.155852
5,3.000000,3.337475,2.000000,4.000000,4.000000,3.311002
6,4.698465,5.000000,5.000000,5.000000,4.870350,5.000000
7,1.000000,1.789695,2.155844,2.000000,2.383400,3.000000
8,3.740899,4.021251,3.835183,4.150897,4.000000,4.000000


## Косинусная-мера

In [ ]:
algo = KNNBaseline(k=5,sim_options= {'name': 'cosine'}, verbose=False)
predictions = algo.fit(trainset).test(testset)
df_unpivot5_cos = df_unpivot.copy()
for i in predictions:
    df_unpivot5_cos = df_unpivot5_cos.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est}, ignore_index=True)
df_unpivot5_cos.pivot(index='userID', columns='itemID', values='rating')

itemID,A,B,C,D,E,F
userID,,,,,,
1,3.000000,4.000000,2.000000,4.000000,1.000000,3.572362
2,3.000000,4.000000,2.000000,4.000000,2.927003,2.000000
3,2.754335,2.000000,5.000000,5.000000,3.270544,5.000000
4,3.134947,4.197382,2.134947,4.134947,4.000000,3.976639
5,3.000000,3.668772,2.000000,4.000000,4.000000,3.714426
6,2.939492,5.000000,5.000000,5.000000,3.360864,5.000000
7,1.000000,3.442211,2.951617,2.000000,2.802029,3.000000
8,2.700881,3.765123,3.231273,4.431273,4.000000,4.000000


## Datalens

В библиотеке есть встроенные Dataset. Можно воспользоваться ими.

Возьмем Dataset movielens (Побробнее о нем https://grouplens.org/datasets/movielens/ )

In [ ]:
data = Dataset.load_builtin('ml-100k')#загружем данные о фильмах

In [ ]:
df = pd.DataFrame(data.raw_ratings)
df.columns = ['user', 'item', 'rating', 'timestamp']
df.head()

,user,item,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [ ]:
df['user'].nunique(), df['item'].nunique()

(943, 1682)

In [ ]:
for i in [1,3,5,7,20]:
    algo = KNNBaseline(k=i, verbose=False) #отключим вывод логирования
    cv=cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)
    print(str(i)+'NN:',np.mean(cv['test_rmse']))

1NN: 1.246339807977332
3NN: 1.0399470383326885
5NN: 0.9917767965869508
7NN: 0.9727265727173074
20NN: 0.9407337899874544
